# Load dataset

In [ ]:
#Acknowlegement: Some codes used to initialise BERT are from BERT tutorial(https://colab.research.google.com/drive/1Y4o3jh3ZH70tl6mCd76vz_IxX23biCPP)

In [ ]:
# Import necessary libraries
import re
import string
import os 
from collections import defaultdict
from collections import Counter
import time
import numpy as np
import torch
import datetime
import copy
from transformers import get_linear_schedule_with_warmup
from transformers import BertForSequenceClassification, AdamW, BertConfig
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from covid_tools import *
from BERTs import *

In [ ]:
import json
import pandas as pd
# load orginal data(COVID-19 TREC-IS 2020 task) from json file.
cv19_dc = pd.read_json('./Pythonbooks/Data/COVID/cv19_dc_test.json',lines=True,orient='records')
cv19_ws = pd.read_json('./Pythonbooks/Data/COVID/cv19_washington_state_test.json',lines=True,orient='records')
cv19_ny = pd.read_json('./Pythonbooks/Data/COVID/cv19_nyc_test.json',lines=True,orient='records')

# Read lablled tweets from json file which contains all labels based on union of labels from all human assessors.
all_labels  = read_union_labels_from_file()

# Merge original dataset with lablled dataset based on id of tweets
cv19_dc_labeled = pd.merge(cv19_dc,all_labels,on=['id'])
cv19_ws_labeled = pd.merge(cv19_ws,all_labels,on=['id'])
cv19_ny_labeled = pd.merge(cv19_ny,all_labels,on=['id'])

# As for priorization task, we use majority vote of labels instead of union.
'''More specifically, if one tweet is assessed by more than two assessors,
 we use the most common label as the priority label. And if one tweet is assessed by less than two assessors, 
 we randomly choose one label as the true label. 
 '''
cv19_dc_labeled = extract_majority_vote_label(cv19_dc_labeled)
cv19_ny_labeled = extract_majority_vote_label(cv19_ny_labeled)
cv19_ws_labeled = extract_majority_vote_label(cv19_ws_labeled)


In [ ]:
def build_trainingset(trainSet):
  # Extract training set from dataFrame
  # Extract features from dataFrame by using feature names
  featureList = ['full_text','entities','favorite_count'] #,'favorited'
  trainF = extract_features(trainSet,featureList)
  # Select column named "priority" as labels
  labelList = ['priority']
  trainL_temp = extract_features(trainSet,labelList)['priority']
  # Extract hashtags from dataFrame for each samples
  trainF = extract_hashtags(trainF)
  
  # Transfer text labels to numerical labels
  trainL = []
  for label in trainL_temp:
    if label == 'Low':
      trainL.append(0)
    elif label == 'Medium':
      trainL.append(1)
    elif label == 'High':
      trainL.append(2)
    elif label == 'Critical':
      trainL.append(3)
  return trainF, trainL

def store_csv(df,location,val_ratio=0.2):
  # Store training, valdation, text set into csv file
  trainF, trainL = build_trainingset(df)
    
  idx = np.arange(df.shape[0])
  np.random.shuffle(idx)
    
  val_size = int(len(idx) * val_ratio)
  if not os.path.exists('priority/'+location):
      os.makedirs('priority/'+location)
    
  print(len(trainL))
  df = pd.DataFrame(trainF)
  df['target'] = trainL
  # Rename labels to target
  df[['full_text','hashtags','favorite_count', 'target']].to_csv('priority/'+location+'/dataset_all.csv',
                   index=False)
    
  df.iloc[idx[val_size:], :][['full_text','hashtags','favorite_count','target']].to_csv('priority/'+location+'/dataset_test.csv',
                   index=False)                 
  df.iloc[idx[val_size:], :][['full_text','hashtags','favorite_count','target']].to_csv(
        'priority/'+location+'/dataset_train.csv',index=False)
    

In [ ]:
# Store training,  test set into csv file
if not os.path.exists('/priority/'):
  store_csv(cv19_ws_labeled,location='WS') 
  store_csv(cv19_dc_labeled,location='DC') 
  store_csv(cv19_ny_labeled,location='NY') 

2577
2504
2502


In [ ]:
# check GPU situation
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

# Pre-process the data 

In [ ]:
# Choose one dataset to use in this section
Tweets = cv19_ws_labeled

In [ ]:
categories = ['GoodsServices','InformationWanted','Volunteer','EmergingThreats','NewSubEvent','ServiceAvailable','Advice',]

# Extract training set
featureList = ['full_text'] #,'favorited'
dataset_inputs = extract_features(Tweets,featureList)

# Extract labels
labelList = ['categories']
labels = extract_features(Tweets,labelList)['categories']

# Transfer a list of text labels into a list of number labels, like [0,1,0,0,0,0,0]
labels = extractLabels(dataset_inputs,labels)


#### Build label for each catergory

In [ ]:
categories = ['GoodsServices','InformationWanted','Volunteer','EmergingThreats','NewSubEvent','ServiceAvailable','Advice']
# Sperate labels for each category and store them in one dict
# A dict Contains labels for all categories e.g. trainL_single[category]
labels_single = {}
for i in range(7):
  print('Currrent is processing on categorie %s'%(categories[i]))
  print()
  # if i == 3:
  #   continue
  labels_one_category = []
  for j in range(len(labels)):
    labels_one_category.append(labels[j][i])
  labels_single[i] = labels_one_category


Currrent is processing on categorie GoodsServices

Currrent is processing on categorie InformationWanted

Currrent is processing on categorie Volunteer

Currrent is processing on categorie EmergingThreats

Currrent is processing on categorie NewSubEvent

Currrent is processing on categorie ServiceAvailable

Currrent is processing on categorie Advice



## Tokenization: Bert tokenizer

In [ ]:
sentences = Tweets.full_text.values
labels = labels

In [ ]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
# Print the original sentence.
print(' Original: ', sentences[0])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(sentences[0]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

Loading BERT tokenizer...
 Original:  I see lots of people panicking about coronavirus around Seattle and King County.

There are already hundreds, if not thousands, of undiagnosed cases in the local area. We will all be exposed to it. The spread will not be stopped. 

I’m not trying to incite panic, though...
Tokenized:  ['i', 'see', 'lots', 'of', 'people', 'panic', '##king', 'about', 'corona', '##virus', 'around', 'seattle', 'and', 'king', 'county', '.', 'there', 'are', 'already', 'hundreds', ',', 'if', 'not', 'thousands', ',', 'of', 'und', '##ia', '##gno', '##sed', 'cases', 'in', 'the', 'local', 'area', '.', 'we', 'will', 'all', 'be', 'exposed', 'to', 'it', '.', 'the', 'spread', 'will', 'not', 'be', 'stopped', '.', 'i', '’', 'm', 'not', 'trying', 'to', 'inc', '##ite', 'panic', ',', 'though', '.', '.', '.']
Token IDs:  [1045, 2156, 7167, 1997, 2111, 6634, 6834, 2055, 21887, 23350, 2105, 5862, 1998, 2332, 2221, 1012, 2045, 2024, 2525, 5606, 1010, 2065, 2025, 5190, 1010, 1997, 6151, 24

In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []

# For every sentence...
for sent in sentences:
    # `encode` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    encoded_sent = tokenizer.encode(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        #max_length = 128,          # Truncate all sentences.
                        #return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.
    # If the length of token is over the bert limitation, cut off the back
    if(len(encoded_sent)>144):
        diff = int((len(encoded_sent) - 145)/2)
        encoded_sent = encoded_sent[diff:144+diff]
    input_ids.append(encoded_sent)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Original:  I see lots of people panicking about coronavirus around Seattle and King County.

There are already hundreds, if not thousands, of undiagnosed cases in the local area. We will all be exposed to it. The spread will not be stopped. 

I’m not trying to incite panic, though...
Token IDs: [101, 1045, 2156, 7167, 1997, 2111, 6634, 6834, 2055, 21887, 23350, 2105, 5862, 1998, 2332, 2221, 1012, 2045, 2024, 2525, 5606, 1010, 2065, 2025, 5190, 1010, 1997, 6151, 2401, 26745, 6924, 3572, 1999, 1996, 2334, 2181, 1012, 2057, 2097, 2035, 2022, 6086, 2000, 2009, 1012, 1996, 3659, 2097, 2025, 2022, 3030, 1012, 1045, 1521, 1049, 2025, 2667, 2000, 4297, 4221, 6634, 1010, 2295, 1012, 1012, 1012, 102]


## Padding


In [ ]:

from keras.preprocessing.sequence import pad_sequences

# Set the maximum sequence length.
MAX_LEN = 144

print('\nPadding/truncating all sentences to %d values...' % MAX_LEN)

print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))

# Pad our input tokens with value 0.
# "post" indicates that we want to pad and truncate at the end of the sequence,
# as opposed to the beginning.
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", 
                          value=0, truncating="post", padding="post")

print('\nDone.')


Padding/truncating all sentences to 144 values...

Padding token: "[PAD]", ID: 0

Done.


## Attension mask

In [ ]:
# Create attention masks
attention_masks = []

# For each sentence...
for sent in input_ids:
    
    # Create the attention mask.
    #   - If a token ID is 0, then it's padding, set the mask to 0.
    #   - If a token ID is > 0, then it's a real token, set the mask to 1.
    att_mask = [int(token_id > 0) for token_id in sent]
    
    # Store the attention mask for this sentence.
    attention_masks.append(att_mask)

# Classication one category by on category

All training split dataset first. Then use BERT to evaluate and store the result into a file.

In [ ]:
# A dict to store all metrics fro each category, Use name of category as the key to search
all_metrics_all_category = {}

### For 'GoodsServices'

#### Split

In [ ]:
# Use train_test_split to split our data into train and validation and test sets for
# training
section_category = 0
labels = labels_single[section_category]
from sklearn.model_selection import train_test_split

# Use 80% for training and 20% for validation.
temp_inputs, test_inputs,  temp_labels, test_labels = train_test_split(input_ids, labels, random_state=1999, test_size=0.2)
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(temp_inputs, temp_labels,random_state=1999, test_size=0.2)

# Do the same for the masks.
temp_masks, test_masks, temp_labels, _ = train_test_split(attention_masks, labels,random_state=1999, test_size=0.2)

train_masks, validation_masks, _, _ = train_test_split(temp_masks, temp_labels,random_state=1999, test_size=0.2)

#### Normal training

In [ ]:
import json
batch_size = 8
lr = 3e-5
test_batch = [test_inputs,test_labels,test_masks]
Bert = Bertnn_info(train_inputs,train_labels,train_masks, validation_inputs,validation_labels,validation_masks,test_batch,batch_size=batch_size,lr=lr)
all_metrics_one, all_tesst_metrics = Bert.searchUpsample(100)
all_metrics_one['test'] = all_tesst_metrics
all_metrics_all_category[section_category] = all_metrics_one
if not os.path.exists('BertSearchResult'):
  os.makedirs('BertSearchResult')

filename = './BertSearchResult/'+str(batch_size) + '_' + str(lr) + '_'  + str(section_category)
with open(filename,'w') as file_obj:
  json.dump(all_metrics_one,file_obj)
  print('Successfully save file %s'%(filename))

del Bert
torch.cuda.empty_cache() 

There are 1 GPU(s) available.
We will use the GPU: GeForce RTX 2070 SUPER


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Current increase 0 times
Current is processing category 1
Before up-sample, there are 1648 samples
There are 3 sampled in this category.
After up-sample, there are 1648 samples

Currrent is processing on categorie GoodsServices

Before up-sample, the ratio of current category and all samples
0
0.0

After up-sample, the ratio of current category and all samples
1645
0.9981796116504854

Currrent is processing on categorie InformationWanted

Before up-sample, the ratio of current category and all samples
0
0.0

After up-sample, the ratio of current category and all samples
3
0.0018203883495145632

(1648, 144)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at


======== Epoch 1 / 4 ========
Training...
  Batch    40  of    206.    Elapsed: 0:00:09.
  Batch    80  of    206.    Elapsed: 0:00:18.
  Batch   120  of    206.    Elapsed: 0:00:27.
  Batch   160  of    206.    Elapsed: 0:00:35.
  Batch   200  of    206.    Elapsed: 0:00:44.

  Average training loss: 0.03
  Accuracy: 0.99
Category: 0
  Precision: 1.00
  Recall: 1.00
  F1: 1.00
Category: 1
  Precision: 0.00
  Recall: 0.00
  F1: 0.00
The average precision is: 0.4991
The average recall is: 0.4982
The average f1 is: 0.4981
  Training epcoh took: 0:00:46

Running Validation...
  Accuracy: 1.00
Category: 0
  Precision: 1.0000
  Recall: 1.0000
  F1: 1.0000
Category: 1
  Precision: 0.0000
  Recall: 0.0000
  F1: 0.0000
The average precision is: 0.5000
The average recall is: 0.5000
The average f1 is: 0.5000
  Validation took: 0:00:02

======== Epoch 2 / 4 ========
Training...
  Batch    40  of    206.    Elapsed: 0:00:09.
  Batch    80  of    206.    Elapsed: 0:00:18.
  Batch   120  of    206.

#### Incremental recitfied training

In [ ]:
import json
batch_size = 32
lr = 3e-5
test_batch = [test_inputs,test_labels,test_masks]
Bert = Bertnn_rectified_info(train_inputs,train_labels,train_masks, validation_inputs,validation_labels,validation_masks,test_batch,batch_size=batch_size,lr=lr)
all_metrics_one, all_tesst_metrics = Bert.searchUpsample(100)
all_metrics_one['test'] = all_tesst_metrics
all_metrics_all_category[section_category] = all_metrics_one
if not os.path.exists('BertSearchResult'):
  os.makedirs('BertSearchResult')

filename = './BertSearchResult/'+'rectified'+'_'+str(section_category)+'_'+str(batch_size) + '_' + str(lr) + '_'  + str(section_category)
with open(filename,'w') as file_obj:
  json.dump(all_metrics_one,file_obj)
  print('Successfully save file %s'%(filename))

del Bert
torch.cuda.empty_cache() 

There are 1 GPU(s) available.
We will use the GPU: TITAN RTX


I0904 07:41:22.816618 139984129095488 configuration_utils.py:264] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0904 07:41:22.817810 139984129095488 configuration_utils.py:300] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

I0904 07:41:24.544897 139984129095488 modeling_utils.py:667] loading weights file https://

Current increase 0 times
Current is processing category 1
Before up-sample, there are 1648 samples
There are 3 sampled in this category.
After up-sample, there are 1648 samples

Currrent is processing on categorie GoodsServices

Before up-sample, the ratio of current category and all samples
0
0.0

After up-sample, the ratio of current category and all samples
1645
0.9981796116504854

Currrent is processing on categorie InformationWanted

Before up-sample, the ratio of current category and all samples
0
0.0

After up-sample, the ratio of current category and all samples
3
0.0018203883495145632

(1648, 144)


I0904 07:41:27.731730 139984129095488 configuration_utils.py:264] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0904 07:41:27.732635 139984129095488 configuration_utils.py:300] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

I0904 07:41:27.822070 139984129095488 modeling_utils.py:667] loading weights file https://


======== Epoch 1 / 4 ========
Training...
Invoke 1
Invoke 1
  Batch    40  of     52.    Elapsed: 0:00:14.
Invoke 1

  Average training loss: 0.38
  Accuracy: 0.99
Category: 0
  Precision: 1.00
  Recall: 0.99
  F1: 1.00
Category: 1
  Precision: 0.00
  Recall: 0.00
  F1: 0.00
The average precision is: 0.4991
The average recall is: 0.4973
The average f1 is: 0.4980
  Training epcoh took: 0:00:18

Running Validation...
  Accuracy: 1.00
Category: 0
  Precision: 1.0000
  Recall: 1.0000
  F1: 1.0000
Category: 1
  Precision: 0.0000
  Recall: 0.0000
  F1: 0.0000
The average precision is: 0.5000
The average recall is: 0.5000
The average f1 is: 0.5000
  Validation took: 0:00:01

======== Epoch 2 / 4 ========
Training...
Invoke 1
Invoke 1
  Batch    40  of     52.    Elapsed: 0:00:14.
Invoke 1

  Average training loss: 0.36
  Accuracy: 1.00
Category: 0
  Precision: 1.00
  Recall: 1.00
  F1: 1.00
Category: 1
  Precision: 0.00
  Recall: 0.00
  F1: 0.00
The average precision is: 0.4991
The average r

...


======== Epoch 1 / 4 ========
Training...
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
  Batch    40  of     59.    Elapsed: 0:00:21.
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1

  Average training loss: 0.28
  Accuracy: 0.96
Category: 0
  Precision: 0.96
  Recall: 0.98
  F1: 0.97
Category: 1
  Precision: 0.80
  Recall: 0.80
  F1: 0.79
The average precision is: 0.8801
The average recall is: 0.8874
The average f1 is: 0.8780
  Training epcoh took: 0:00:31

Running Validation...
  Accuracy: 1.00
Category: 0
  Precision: 1.0000
  Recall: 1.0000
  F1: 1.00

I0904 07:57:22.757156 139984129095488 configuration_utils.py:264] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0904 07:57:22.758334 139984129095488 configuration_utils.py:300] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

I0904 07:57:22.798760 139984129095488 modeling_utils.py:667] loading weights file https://


======== Epoch 1 / 4 ========
Training...
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
  Batch    40  of     59.    Elapsed: 0:00:21.
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1

  Average training loss: 0.27
  Accuracy: 0.98
Category: 0
  Precision: 0.98
  Recall: 1.00
  F1: 0.99
Category: 1
  Precision: 0.85
  Recall: 0.82
  F1: 0.83
The average precision is: 0.9135
The average recall is: 0.9110
The average f1 is: 0.9082
  Training epcoh took: 0:00:31

Running Validation...
  Accuracy: 1.00
Category: 0
  Precision: 1.0000
  Recall: 1.0000
  F1: 1.0000
Catego

I0904 07:59:37.952741 139984129095488 configuration_utils.py:264] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0904 07:59:37.953974 139984129095488 configuration_utils.py:300] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

I0904 07:59:38.000062 139984129095488 modeling_utils.py:667] loading weights file https://


======== Epoch 1 / 4 ========
Training...
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
  Batch    40  of     60.    Elapsed: 0:00:21.
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1

  Average training loss: 0.30
  Accuracy: 0.97
Category: 0
  Precision: 0.97
  Recall: 0.99
  F1: 0.98
Category: 1
  Precision: 0.91
  Recall: 0.80
  F1: 0.82
The average precision is: 0.9402
The average recall is: 0.8956
The average f1 is: 0.9008
  Training epcoh took: 0:00:32

Running Validation...
  Accuracy: 1.00
Category: 0
  Precision: 1.0000
  Recall: 1.0000
 

In [ ]:
all_metrics_one['test']

{0: [[{0: 0.9980769230769231, 1: 0.0}],
  [{0: 1.0, 1: 0.0}],
  [{0: 0.998974358974359, 1: 0.0}],
  [0.9980769230769231]]}

### For 'InformationWanted'

#### Split

In [ ]:
# Use train_test_split to split our data into train and validation and test sets for
# training
section_category = 0
labels = labels_single[section_category]
from sklearn.model_selection import train_test_split

# Use 80% for training and 20% for validation.
temp_inputs, test_inputs,  temp_labels, test_labels = train_test_split(input_ids, labels, random_state=1999, test_size=0.2)
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(temp_inputs, temp_labels,random_state=1999, test_size=0.2)

# Do the same for the masks.
temp_masks, test_masks, temp_labels, _ = train_test_split(attention_masks, labels,random_state=1999, test_size=0.2)

train_masks, validation_masks, _, _ = train_test_split(temp_masks, temp_labels,random_state=1999, test_size=0.2)

#### Normal training

In [ ]:
import json
batch_size = 16
lr = 2e-5
test_batch = [test_inputs,test_labels,test_masks]
Bert = Bertnn_info(train_inputs,train_labels,train_masks, validation_inputs,validation_labels,validation_masks,test_batch,batch_size=batch_size,lr=lr)
all_metrics_one, all_tesst_metrics = Bert.searchUpsample(100)
all_metrics_one['test'] = all_tesst_metrics
all_metrics_all_category[section_category] = all_metrics_one
if not os.path.exists('BertSearchResult'):
  os.makedirs('BertSearchResult')

filename = './BertSearchResult/'+str(section_category)+'_'+str(batch_size) + '_' + str(lr) + '_'  + str(section_category)
with open(filename,'w') as file_obj:
  json.dump(all_metrics_one,file_obj)
  print('Successfully save file %s'%(filename))

del Bert
torch.cuda.empty_cache() 

There are 1 GPU(s) available.
We will use the GPU: TITAN RTX


I0903 01:37:20.923159 140575546472256 configuration_utils.py:264] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0903 01:37:20.924405 140575546472256 configuration_utils.py:300] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

I0903 01:37:21.036392 140575546472256 modeling_utils.py:667] loading weights file https://

Current increase 0 times
Current is processing category 1
Before up-sample, there are 1648 samples
There are 7 sampled in this category.
After up-sample, there are 1648 samples

Currrent is processing on categorie GoodsServices

Before up-sample, the ratio of current category and all samples
0
0.0

After up-sample, the ratio of current category and all samples
1641
0.995752427184466

Currrent is processing on categorie InformationWanted

Before up-sample, the ratio of current category and all samples
0
0.0

After up-sample, the ratio of current category and all samples
7
0.00424757281553398

(1648, 144)


I0903 01:37:24.331399 140575546472256 configuration_utils.py:264] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0903 01:37:24.333372 140575546472256 configuration_utils.py:300] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

I0903 01:37:24.364717 140575546472256 modeling_utils.py:667] loading weights file https://


======== Epoch 1 / 4 ========
Training...
  Batch    40  of    103.    Elapsed: 0:00:08.
  Batch    80  of    103.    Elapsed: 0:00:15.

  Average training loss: 0.06
  Accuracy: 0.98
Category: 0
  Precision: 0.99
  Recall: 0.99
  F1: 0.98
Category: 1
  Precision: 0.00
  Recall: 0.00
  F1: 0.00
The average precision is: 0.4930
The average recall is: 0.4927
The average f1 is: 0.4924
  Training epcoh took: 0:00:20

Running Validation...
  Accuracy: 0.99
Category: 0
  Precision: 0.9928
  Recall: 1.0000
  F1: 0.9963
Category: 1
  Precision: 0.0000
  Recall: 0.0000
  F1: 0.0000
The average precision is: 0.4964
The average recall is: 0.5000
The average f1 is: 0.4981
  Validation took: 0:00:01

======== Epoch 2 / 4 ========
Training...
  Batch    40  of    103.    Elapsed: 0:00:08.
  Batch    80  of    103.    Elapsed: 0:00:15.

  Average training loss: 0.03
  Accuracy: 1.00
Category: 0
  Precision: 1.00
  Recall: 1.00
  F1: 1.00
Category: 1
  Precision: 0.00
  Recall: 0.00
  F1: 0.00
The av

...


======== Epoch 1 / 4 ========
Training...
  Batch    40  of    134.    Elapsed: 0:00:09.
  Batch    80  of    134.    Elapsed: 0:00:18.
  Batch   120  of    134.    Elapsed: 0:00:27.

  Average training loss: 0.09
  Accuracy: 0.97
Category: 0
  Precision: 0.97
  Recall: 0.99
  F1: 0.98
Category: 1
  Precision: 0.92
  Recall: 0.89
  F1: 0.90
The average precision is: 0.9457
The average recall is: 0.9398
The average f1 is: 0.9371
  Training epcoh took: 0:00:30

Running Validation...
  Accuracy: 0.99
Category: 0
  Precision: 0.9922
  Recall: 1.0000
  F1: 0.9960
Category: 1
  Precision: 0.0000
  Recall: 0.0000
  F1: 0.0000
The average precision is: 0.4961
The average recall is: 0.5000
The average f1 is: 0.4980
  Validation took: 0:00:01

======== Epoch 2 / 4 ========
Training...
  Batch    40  of    134.    Elapsed: 0:00:09.
  Batch    80  of    134.    Elapsed: 0:00:18.
  Batch   120  of    134.    Elapsed: 0:00:27.

  Average training loss: 0.00
  Accuracy: 1.00
Category: 0
  Precision:

I0903 01:52:46.136069 140575546472256 configuration_utils.py:264] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0903 01:52:46.137539 140575546472256 configuration_utils.py:300] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

I0903 01:52:46.232057 140575546472256 modeling_utils.py:667] loading weights file https://


======== Epoch 1 / 4 ========
Training...
  Batch    40  of    138.    Elapsed: 0:00:09.
  Batch    80  of    138.    Elapsed: 0:00:18.
  Batch   120  of    138.    Elapsed: 0:00:27.

  Average training loss: 0.09
  Accuracy: 0.97
Category: 0
  Precision: 0.97
  Recall: 1.00
  F1: 0.98
Category: 1
  Precision: 0.94
  Recall: 0.91
  F1: 0.92
The average precision is: 0.9566
The average recall is: 0.9537
The average f1 is: 0.9516
  Training epcoh took: 0:00:31

Running Validation...
  Accuracy: 0.99
Category: 0
  Precision: 0.9928
  Recall: 1.0000
  F1: 0.9963
Category: 1
  Precision: 0.0000
  Recall: 0.0000
  F1: 0.0000
The average precision is: 0.4964
The average recall is: 0.5000
The average f1 is: 0.4981
  Validation took: 0:00:01

======== Epoch 2 / 4 ========
Training...
  Batch    40  of    138.    Elapsed: 0:00:09.
  Batch    80  of    138.    Elapsed: 0:00:18.
  Batch   120  of    138.    Elapsed: 0:00:27.

  Average training loss: 0.00
  Accuracy: 1.00
Category: 0
  Precision:

I0903 01:55:01.120578 140575546472256 configuration_utils.py:264] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0903 01:55:01.122064 140575546472256 configuration_utils.py:300] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

I0903 01:55:01.178817 140575546472256 modeling_utils.py:667] loading weights file https://


======== Epoch 1 / 4 ========
Training...
  Batch    40  of    143.    Elapsed: 0:00:09.
  Batch    80  of    143.    Elapsed: 0:00:18.
  Batch   120  of    143.    Elapsed: 0:00:27.

  Average training loss: 0.10
  Accuracy: 0.97
Category: 0
  Precision: 0.96
  Recall: 0.99
  F1: 0.97
Category: 1
  Precision: 0.94
  Recall: 0.91
  F1: 0.91
The average precision is: 0.9542
The average recall is: 0.9463
The average f1 is: 0.9428
  Training epcoh took: 0:00:33

Running Validation...
  Accuracy: 0.99
Category: 0
  Precision: 0.9928
  Recall: 1.0000
  F1: 0.9963
Category: 1
  Precision: 0.0000
  Recall: 0.0000
  F1: 0.0000
The average precision is: 0.4964
The average recall is: 0.5000
The average f1 is: 0.4981
  Validation took: 0:00:01

======== Epoch 2 / 4 ========
Training...
  Batch    40  of    143.    Elapsed: 0:00:09.
  Batch    80  of    143.    Elapsed: 0:00:18.
  Batch   120  of    143.    Elapsed: 0:00:27.

  Average training loss: 0.00
  Accuracy: 1.00
Category: 0
  Precision:

In [ ]:
all_metrics_one

{0: defaultdict(None,
             {'train_loss_values': [0.02451061978212838,
               0.02451061978212838,
               0.014020035567792874,
               0.014020035567792874,
               0.014816705897016433,
               0.014816705897016433,
               0.012088468786582205,
               0.012088468786582205],
              'train_acc_all': [0.9981796116504854,
               0.9981796116504854,
               0.9981796116504854,
               0.9981796116504854],
              'train_precision_all': [{0: 0.9981796116504854, 1: 0.0},
               {0: 0.9981796116504854, 1: 0.0},
               {0: 0.9981796116504854, 1: 0.0},
               {0: 0.9981796116504854, 1: 0.0}],
              'train_recall_all': [{0: 1.0, 1: 0.0},
               {0: 1.0, 1: 0.0},
               {0: 1.0, 1: 0.0},
               {0: 1.0, 1: 0.0}],
              'train_f1_all': [{0: 0.9990291262135923, 1: 0.0},
               {0: 0.9990291262135923, 1: 0.0},
               {0: 0.99

#### Incremental recitfied training

In [ ]:
import json
batch_size = 32
lr = 3e-5
test_batch = [test_inputs,test_labels,test_masks]
Bert = Bertnn_rectified_info(train_inputs,train_labels,train_masks, validation_inputs,validation_labels,validation_masks,test_batch,batch_size=batch_size,lr=lr)
all_metrics_one, all_tesst_metrics = Bert.searchUpsample(100)
all_metrics_one['test'] = all_tesst_metrics
all_metrics_all_category[section_category] = all_metrics_one
if not os.path.exists('BertSearchResult'):
  os.makedirs('BertSearchResult')

filename = './BertSearchResult/'+'rectified'+'_'+str(section_category)+'_'+str(batch_size) + '_' + str(lr) + '_'  + str(section_category)
with open(filename,'w') as file_obj:
  json.dump(all_metrics_one,file_obj)
  print('Successfully save file %s'%(filename))

del Bert
torch.cuda.empty_cache() 

There are 1 GPU(s) available.
We will use the GPU: TITAN RTX


I0904 13:53:40.984838 139984129095488 configuration_utils.py:264] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0904 13:53:40.985996 139984129095488 configuration_utils.py:300] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

I0904 13:53:41.077767 139984129095488 modeling_utils.py:667] loading weights file https://

Current increase 0 times
Current is processing category 1
Before up-sample, there are 1648 samples
There are 7 sampled in this category.
After up-sample, there are 1648 samples

Currrent is processing on categorie GoodsServices

Before up-sample, the ratio of current category and all samples
0
0.0

After up-sample, the ratio of current category and all samples
1641
0.995752427184466

Currrent is processing on categorie InformationWanted

Before up-sample, the ratio of current category and all samples
0
0.0

After up-sample, the ratio of current category and all samples
7
0.00424757281553398

(1648, 144)


I0904 13:53:44.402355 139984129095488 configuration_utils.py:264] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0904 13:53:44.403277 139984129095488 configuration_utils.py:300] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

I0904 13:53:44.436729 139984129095488 modeling_utils.py:667] loading weights file https://


======== Epoch 1 / 4 ========
Training...
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
  Batch    40  of     52.    Elapsed: 0:00:14.
Invoke 1

  Average training loss: 0.61
  Accuracy: 0.99
Category: 0
  Precision: 1.00
  Recall: 0.99
  F1: 0.99
Category: 1
  Precision: 0.00
  Recall: 0.00
  F1: 0.00
The average precision is: 0.4979
The average recall is: 0.4964
The average f1 is: 0.4967
  Training epcoh took: 0:00:18

Running Validation...
  Accuracy: 0.99
Category: 0
  Precision: 0.9928
  Recall: 1.0000
  F1: 0.9963
Category: 1
  Precision: 0.0000
  Recall: 0.0000
  F1: 0.0000
The average precision is: 0.4964
The average recall is: 0.5000
The average f1 is: 0.4982
  Validation took: 0:00:01

======== Epoch 2 / 4 ========
Training...
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
  Batch    40  of     52.    Elapsed: 0:00:14.
Invoke 1

  Average training loss: 0.63
  Accuracy: 1.00
Category: 0
  Precision: 1.00
  Recall: 1.00
  F1: 1.00
Category: 1
  Precision: 0.00
  Recall:

...


======== Epoch 1 / 4 ========
Training...
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
  Batch    40  of     59.    Elapsed: 0:00:22.
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1

  Average training loss: 0.53
  Accuracy: 0.95
Category: 0
  Precision: 0.97
  Recall: 0.98
  F1: 0.96
Category: 1
  Precision: 0.75
  Recall: 0.68
  F1: 0.68
The average precision is: 0.8560
The average recall is: 0.8298
The average f1 is: 0.8233
  Training epcoh took: 0:00:32

Running Validation...
  Accuracy: 0.99
Category: 0
  Precision: 0.9928
  Recall: 1.0000
  F1: 0.9963
Catego

I0904 14:01:34.024106 139984129095488 configuration_utils.py:264] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0904 14:01:34.025238 139984129095488 configuration_utils.py:300] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

I0904 14:01:34.075600 139984129095488 modeling_utils.py:667] loading weights file https://


======== Epoch 1 / 4 ========
Training...
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
  Batch    40  of     61.    Elapsed: 0:00:23.
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1

  Average training loss: 0.28
  Accuracy: 0.95
Category: 0
  Precision: 0.96
  Recall: 0.96
  F1: 0.96
Category: 1
  Precision: 0.87
  Recall: 0.87
  F1: 0.85
The average precision is: 0.9117
The average recall is: 0.9171
The average f1 is: 0.9044
  Training epcoh took: 0:00:35

Running Validation...
  Accuracy: 0.99
Category: 0
  Precision: 0.9925
  Recall:

I0904 14:03:59.433241 139984129095488 configuration_utils.py:264] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0904 14:03:59.434335 139984129095488 configuration_utils.py:300] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

I0904 14:03:59.490442 139984129095488 modeling_utils.py:667] loading weights file https://


======== Epoch 1 / 4 ========
Training...
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
  Batch    40  of     63.    Elapsed: 0:00:22.
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1
Invoke 1

  Average training loss: 0.29
  Accuracy: 0.97
Category: 0
  Precision: 0.97
  Recall: 1.00
  F1: 0.98
Category: 1
  Precision: 0.85
  Recall: 0.83
  F1: 0.84
The average precision is: 0.9114
The average recall is: 0.9149
The average f1 is: 0.9094
  Training epcoh took: 0:00:34

Running Validation...
  Accuracy: 0.99
Category: 0
  Precision

In [ ]:
all_metrics_one

{0: defaultdict(None,
             {'train_loss_values': [0.02451061978212838,
               0.02451061978212838,
               0.014020035567792874,
               0.014020035567792874,
               0.014816705897016433,
               0.014816705897016433,
               0.012088468786582205,
               0.012088468786582205],
              'train_acc_all': [0.9981796116504854,
               0.9981796116504854,
               0.9981796116504854,
               0.9981796116504854],
              'train_precision_all': [{0: 0.9981796116504854, 1: 0.0},
               {0: 0.9981796116504854, 1: 0.0},
               {0: 0.9981796116504854, 1: 0.0},
               {0: 0.9981796116504854, 1: 0.0}],
              'train_recall_all': [{0: 1.0, 1: 0.0},
               {0: 1.0, 1: 0.0},
               {0: 1.0, 1: 0.0},
               {0: 1.0, 1: 0.0}],
              'train_f1_all': [{0: 0.9990291262135923, 1: 0.0},
               {0: 0.9990291262135923, 1: 0.0},
               {0: 0.99

### For 'Volunteer'

#### Split

In [ ]:
# Use train_test_split to split our data into train and validation and test sets for
# training
section_category = 0
labels = labels_single[section_category]
from sklearn.model_selection import train_test_split

# Use 80% for training and 20% for validation.
temp_inputs, test_inputs,  temp_labels, test_labels = train_test_split(input_ids, labels, random_state=1999, test_size=0.2)
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(temp_inputs, temp_labels,random_state=1999, test_size=0.2)

# Do the same for the masks.
temp_masks, test_masks, temp_labels, _ = train_test_split(attention_masks, labels,random_state=1999, test_size=0.2)

train_masks, validation_masks, _, _ = train_test_split(temp_masks, temp_labels,random_state=1999, test_size=0.2)

#### Normal training

In [ ]:
import json
batch_size = 16
lr = 2e-5
test_batch = [test_inputs,test_labels,test_masks]
Bert = Bertnn_info(train_inputs,train_labels,train_masks, validation_inputs,validation_labels,validation_masks,test_batch,batch_size=batch_size,lr=lr)
all_metrics_one, all_tesst_metrics = Bert.searchUpsample(100)
all_metrics_one['test'] = all_tesst_metrics
all_metrics_all_category[section_category] = all_metrics_one
if not os.path.exists('BertSearchResult'):
  os.makedirs('BertSearchResult')

filename = './BertSearchResult/'+str(section_category)+'_'+str(batch_size) + '_' + str(lr) + '_'  + str(section_category)
with open(filename,'w') as file_obj:
  json.dump(all_metrics_one,file_obj)
  print('Successfully save file %s'%(filename))

del Bert
torch.cuda.empty_cache() 

There are 1 GPU(s) available.
We will use the GPU: TITAN RTX


I0903 05:49:34.023838 140575546472256 configuration_utils.py:264] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0903 05:49:34.024934 140575546472256 configuration_utils.py:300] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

I0903 05:49:35.389530 140575546472256 modeling_utils.py:667] loading weights file https://

Current increase 0 times
Current is processing category 1
Before up-sample, there are 1648 samples
There are 13 sampled in this category.
After up-sample, there are 1648 samples

Currrent is processing on categorie GoodsServices

Before up-sample, the ratio of current category and all samples
0
0.0

After up-sample, the ratio of current category and all samples
1635
0.9921116504854369

Currrent is processing on categorie InformationWanted

Before up-sample, the ratio of current category and all samples
0
0.0

After up-sample, the ratio of current category and all samples
13
0.007888349514563107

(1648, 144)


I0903 05:49:38.730350 140575546472256 configuration_utils.py:264] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0903 05:49:38.731376 140575546472256 configuration_utils.py:300] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

I0903 05:49:39.215317 140575546472256 modeling_utils.py:667] loading weights file https://


======== Epoch 1 / 4 ========
Training...
  Batch    40  of    103.    Elapsed: 0:00:07.
  Batch    80  of    103.    Elapsed: 0:00:15.

  Average training loss: 0.09
  Accuracy: 0.98
Category: 0
  Precision: 0.99
  Recall: 0.99
  F1: 0.99
Category: 1
  Precision: 0.00
  Recall: 0.00
  F1: 0.00
The average precision is: 0.4947
The average recall is: 0.4946
The average f1 is: 0.4937
  Training epcoh took: 0:00:19

Running Validation...
  Accuracy: 0.99
Category: 0
  Precision: 0.9904
  Recall: 1.0000
  F1: 0.9950
Category: 1
  Precision: 0.0000
  Recall: 0.0000
  F1: 0.0000
The average precision is: 0.4952
The average recall is: 0.5000
The average f1 is: 0.4975
  Validation took: 0:00:01

======== Epoch 2 / 4 ========
Training...
  Batch    40  of    103.    Elapsed: 0:00:08.
  Batch    80  of    103.    Elapsed: 0:00:15.

  Average training loss: 0.04
  Accuracy: 0.99
Category: 0
  Precision: 0.99
  Recall: 1.00
  F1: 1.00
Category: 1
  Precision: 0.00
  Recall: 0.00
  F1: 0.00
The av

...


======== Epoch 1 / 4 ========
Training...
  Batch    40  of    160.    Elapsed: 0:00:09.
  Batch    80  of    160.    Elapsed: 0:00:18.
  Batch   120  of    160.    Elapsed: 0:00:27.

  Average training loss: 0.10
  Accuracy: 0.97
Category: 0
  Precision: 0.96
  Recall: 0.99
  F1: 0.98
Category: 1
  Precision: 0.95
  Recall: 0.91
  F1: 0.92
The average precision is: 0.9560
The average recall is: 0.9540
The average f1 is: 0.9502
  Training epcoh took: 0:00:35

Running Validation...
  Accuracy: 0.99
Category: 0
  Precision: 0.9904
  Recall: 1.0000
  F1: 0.9950
Category: 1
  Precision: 0.0000
  Recall: 0.0000
  F1: 0.0000
The average precision is: 0.4952
The average recall is: 0.5000
The average f1 is: 0.4975
  Validation took: 0:00:01

======== Epoch 2 / 4 ========
Training...
  Batch    40  of    160.    Elapsed: 0:00:09.
  Batch    80  of    160.    Elapsed: 0:00:19.
  Batch   120  of    160.    Elapsed: 0:00:29.

  Average training loss: 0.00
  Accuracy: 1.00
Category: 0
  Precision:

I0903 06:06:34.006869 140575546472256 configuration_utils.py:264] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0903 06:06:34.007946 140575546472256 configuration_utils.py:300] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

I0903 06:06:34.049268 140575546472256 modeling_utils.py:667] loading weights file https://


======== Epoch 1 / 4 ========
Training...
  Batch    40  of    168.    Elapsed: 0:00:09.
  Batch    80  of    168.    Elapsed: 0:00:19.
  Batch   120  of    168.    Elapsed: 0:00:28.
  Batch   160  of    168.    Elapsed: 0:00:37.

  Average training loss: 0.10
  Accuracy: 0.97
Category: 0
  Precision: 0.97
  Recall: 0.99
  F1: 0.98
Category: 1
  Precision: 0.98
  Recall: 0.95
  F1: 0.95
The average precision is: 0.9731
The average recall is: 0.9661
The average f1 is: 0.9649
  Training epcoh took: 0:00:39

Running Validation...
  Accuracy: 0.99
Category: 0
  Precision: 0.9904
  Recall: 0.9976
  F1: 0.9938
Category: 1
  Precision: 0.0000
  Recall: 0.0000
  F1: 0.0000
The average precision is: 0.4952
The average recall is: 0.4988
The average f1 is: 0.4969
  Validation took: 0:00:01

======== Epoch 2 / 4 ========
Training...
  Batch    40  of    168.    Elapsed: 0:00:09.
  Batch    80  of    168.    Elapsed: 0:00:19.
  Batch   120  of    168.    Elapsed: 0:00:28.
  Batch   160  of    168.

I0903 06:09:25.052129 140575546472256 configuration_utils.py:264] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0903 06:09:25.053167 140575546472256 configuration_utils.py:300] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

I0903 06:09:26.375674 140575546472256 modeling_utils.py:667] loading weights file https://


======== Epoch 1 / 4 ========
Training...
  Batch    40  of    177.    Elapsed: 0:00:09.
  Batch    80  of    177.    Elapsed: 0:00:18.
  Batch   120  of    177.    Elapsed: 0:00:28.
  Batch   160  of    177.    Elapsed: 0:00:38.

  Average training loss: 0.09
  Accuracy: 0.97
Category: 0
  Precision: 0.98
  Recall: 0.98
  F1: 0.98
Category: 1
  Precision: 0.97
  Recall: 0.96
  F1: 0.96
The average precision is: 0.9725
The average recall is: 0.9737
The average f1 is: 0.9714
  Training epcoh took: 0:00:42

Running Validation...
  Accuracy: 0.99
Category: 0
  Precision: 0.9904
  Recall: 1.0000
  F1: 0.9950
Category: 1
  Precision: 0.0000
  Recall: 0.0000
  F1: 0.0000
The average precision is: 0.4952
The average recall is: 0.5000
The average f1 is: 0.4975
  Validation took: 0:00:02

======== Epoch 2 / 4 ========
Training...
  Batch    40  of    177.    Elapsed: 0:00:10.
  Batch    80  of    177.    Elapsed: 0:00:19.
  Batch   120  of    177.    Elapsed: 0:00:29.
  Batch   160  of    177.

In [ ]:
all_metrics_one

{0: defaultdict(None,
             {'train_loss_values': [0.02451061978212838,
               0.02451061978212838,
               0.014020035567792874,
               0.014020035567792874,
               0.014816705897016433,
               0.014816705897016433,
               0.012088468786582205,
               0.012088468786582205],
              'train_acc_all': [0.9981796116504854,
               0.9981796116504854,
               0.9981796116504854,
               0.9981796116504854],
              'train_precision_all': [{0: 0.9981796116504854, 1: 0.0},
               {0: 0.9981796116504854, 1: 0.0},
               {0: 0.9981796116504854, 1: 0.0},
               {0: 0.9981796116504854, 1: 0.0}],
              'train_recall_all': [{0: 1.0, 1: 0.0},
               {0: 1.0, 1: 0.0},
               {0: 1.0, 1: 0.0},
               {0: 1.0, 1: 0.0}],
              'train_f1_all': [{0: 0.9990291262135923, 1: 0.0},
               {0: 0.9990291262135923, 1: 0.0},
               {0: 0.99

#### Incremental recitfied training

In [ ]:
import json
batch_size = 32
lr = 3e-5
test_batch = [test_inputs,test_labels,test_masks]
Bert = Bertnn_rectified_info(train_inputs,train_labels,train_masks, validation_inputs,validation_labels,validation_masks,test_batch,batch_size=batch_size,lr=lr)
all_metrics_one, all_tesst_metrics = Bert.searchUpsample(100)
all_metrics_one['test'] = all_tesst_metrics
all_metrics_all_category[section_category] = all_metrics_one
if not os.path.exists('BertSearchResult'):
  os.makedirs('BertSearchResult')

filename = './BertSearchResult/'+'rectified'+'_'+str(section_category)+'_'+str(batch_size) + '_' + str(lr) + '_'  + str(section_category)
with open(filename,'w') as file_obj:
  json.dump(all_metrics_one,file_obj)
  print('Successfully save file %s'%(filename))

del Bert
torch.cuda.empty_cache() 

In [ ]:
all_metrics_one['test']

{0: [[{0: 0.9980769230769231, 1: 0.0}],
  [{0: 1.0, 1: 0.0}],
  [{0: 0.998974358974359, 1: 0.0}],
  [0.9980769230769231]]}

### For 'EmergingThreats'

#### Split

In [ ]:
# Use train_test_split to split our data into train and validation and test sets for
# training
section_category = 0
labels = labels_single[section_category]
from sklearn.model_selection import train_test_split

# Use 80% for training and 20% for validation.
temp_inputs, test_inputs,  temp_labels, test_labels = train_test_split(input_ids, labels, random_state=1999, test_size=0.2)
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(temp_inputs, temp_labels,random_state=1999, test_size=0.2)

# Do the same for the masks.
temp_masks, test_masks, temp_labels, _ = train_test_split(attention_masks, labels,random_state=1999, test_size=0.2)

train_masks, validation_masks, _, _ = train_test_split(temp_masks, temp_labels,random_state=1999, test_size=0.2)

#### Normal training

In [ ]:
import json
batch_size = 16
lr = 2e-5
test_batch = [test_inputs,test_labels,test_masks]
Bert = Bertnn_info(train_inputs,train_labels,train_masks, validation_inputs,validation_labels,validation_masks,test_batch,batch_size=batch_size,lr=lr)
all_metrics_one, all_tesst_metrics = Bert.searchUpsample(100)
all_metrics_one['test'] = all_tesst_metrics
all_metrics_all_category[section_category] = all_metrics_one
if not os.path.exists('BertSearchResult'):
  os.makedirs('BertSearchResult')

filename = './BertSearchResult/'+str(section_category)+'_'+str(batch_size) + '_' + str(lr) + '_'  + str(section_category)
with open(filename,'w') as file_obj:
  json.dump(all_metrics_one,file_obj)
  print('Successfully save file %s'%(filename))

del Bert
torch.cuda.empty_cache() 

There are 1 GPU(s) available.
We will use the GPU: TITAN RTX


I0903 12:00:24.529457 140575546472256 configuration_utils.py:264] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0903 12:00:24.530456 140575546472256 configuration_utils.py:300] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

I0903 12:00:24.646162 140575546472256 modeling_utils.py:667] loading weights file https://

Current increase 0 times
Current is processing category 1
Before up-sample, there are 1648 samples
There are 0 sampled in this category.
After up-sample, there are 1648 samples

Currrent is processing on categorie GoodsServices

Before up-sample, the ratio of current category and all samples
0
0.0

After up-sample, the ratio of current category and all samples
1648
1.0

Currrent is processing on categorie InformationWanted

Before up-sample, the ratio of current category and all samples
0
0.0

After up-sample, the ratio of current category and all samples
0
0.0

(1648, 144)


I0903 12:00:27.709919 140575546472256 configuration_utils.py:264] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0903 12:00:27.710898 140575546472256 configuration_utils.py:300] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

I0903 12:00:27.744937 140575546472256 modeling_utils.py:667] loading weights file https://


======== Epoch 1 / 4 ========
Training...
  Batch    40  of    103.    Elapsed: 0:00:07.
  Batch    80  of    103.    Elapsed: 0:00:15.

  Average training loss: 0.03
  Accuracy: 1.00
Category: 0
  Precision: 1.00
  Recall: 1.00
  F1: 1.00
Category: 1
  Precision: 0.00
  Recall: 0.00
  F1: 0.00
The average precision is: 0.5000
The average recall is: 0.5000
The average f1 is: 0.5000
  Training epcoh took: 0:00:19

Running Validation...
  Accuracy: 1.00
Category: 0
  Precision: 1.0000
  Recall: 1.0000
  F1: 1.0000
Category: 1
  Precision: 0.0000
  Recall: 0.0000
  F1: 0.0000
The average precision is: 0.5000
The average recall is: 0.5000
The average f1 is: 0.5000
  Validation took: 0:00:01

======== Epoch 2 / 4 ========
Training...
  Batch    40  of    103.    Elapsed: 0:00:07.
  Batch    80  of    103.    Elapsed: 0:00:15.

  Average training loss: 0.00
  Accuracy: 1.00
Category: 0
  Precision: 1.00
  Recall: 1.00
  F1: 1.00
Category: 1
  Precision: 0.00
  Recall: 0.00
  F1: 0.00
The av

...


======== Epoch 1 / 4 ========
Training...
  Batch    40  of    103.    Elapsed: 0:00:09.
  Batch    80  of    103.    Elapsed: 0:00:18.

  Average training loss: 0.05
  Accuracy: 0.99
Category: 0
  Precision: 1.00
  Recall: 0.99
  F1: 0.99
Category: 1
  Precision: 0.00
  Recall: 0.00
  F1: 0.00
The average precision is: 0.5000
The average recall is: 0.4933
The average f1 is: 0.4944
  Training epcoh took: 0:00:23

Running Validation...
  Accuracy: 1.00
Category: 0
  Precision: 1.0000
  Recall: 1.0000
  F1: 1.0000
Category: 1
  Precision: 0.0000
  Recall: 0.0000
  F1: 0.0000
The average precision is: 0.5000
The average recall is: 0.5000
The average f1 is: 0.5000
  Validation took: 0:00:01

======== Epoch 2 / 4 ========
Training...
  Batch    40  of    103.    Elapsed: 0:00:09.
  Batch    80  of    103.    Elapsed: 0:00:18.

  Average training loss: 0.00
  Accuracy: 1.00
Category: 0
  Precision: 1.00
  Recall: 1.00
  F1: 1.00
Category: 1
  Precision: 0.00
  Recall: 0.00
  F1: 0.00
The av

I0903 12:13:35.874093 140575546472256 configuration_utils.py:264] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0903 12:13:35.875189 140575546472256 configuration_utils.py:300] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

I0903 12:13:35.923444 140575546472256 modeling_utils.py:667] loading weights file https://


======== Epoch 1 / 4 ========
Training...
  Batch    40  of    103.    Elapsed: 0:00:09.
  Batch    80  of    103.    Elapsed: 0:00:18.

  Average training loss: 0.05
  Accuracy: 0.99
Category: 0
  Precision: 1.00
  Recall: 0.99
  F1: 0.99
Category: 1
  Precision: 0.00
  Recall: 0.00
  F1: 0.00
The average precision is: 0.5000
The average recall is: 0.4930
The average f1 is: 0.4945
  Training epcoh took: 0:00:23

Running Validation...
  Accuracy: 1.00
Category: 0
  Precision: 1.0000
  Recall: 1.0000
  F1: 1.0000
Category: 1
  Precision: 0.0000
  Recall: 0.0000
  F1: 0.0000
The average precision is: 0.5000
The average recall is: 0.5000
The average f1 is: 0.5000
  Validation took: 0:00:01

======== Epoch 2 / 4 ========
Training...
  Batch    40  of    103.    Elapsed: 0:00:09.
  Batch    80  of    103.    Elapsed: 0:00:18.

  Average training loss: 0.00
  Accuracy: 1.00
Category: 0
  Precision: 1.00
  Recall: 1.00
  F1: 1.00
Category: 1
  Precision: 0.00
  Recall: 0.00
  F1: 0.00
The av

I0903 12:15:19.170035 140575546472256 configuration_utils.py:264] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0903 12:15:19.171057 140575546472256 configuration_utils.py:300] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

I0903 12:15:19.226593 140575546472256 modeling_utils.py:667] loading weights file https://


======== Epoch 1 / 4 ========
Training...
  Batch    40  of    103.    Elapsed: 0:00:09.
  Batch    80  of    103.    Elapsed: 0:00:18.

  Average training loss: 0.04
  Accuracy: 0.99
Category: 0
  Precision: 1.00
  Recall: 0.99
  F1: 0.99
Category: 1
  Precision: 0.00
  Recall: 0.00
  F1: 0.00
The average precision is: 0.5000
The average recall is: 0.4939
The average f1 is: 0.4951
  Training epcoh took: 0:00:23

Running Validation...
  Accuracy: 1.00
Category: 0
  Precision: 1.0000
  Recall: 1.0000
  F1: 1.0000
Category: 1
  Precision: 0.0000
  Recall: 0.0000
  F1: 0.0000
The average precision is: 0.5000
The average recall is: 0.5000
The average f1 is: 0.5000
  Validation took: 0:00:01

======== Epoch 2 / 4 ========
Training...
  Batch    40  of    103.    Elapsed: 0:00:09.
  Batch    80  of    103.    Elapsed: 0:00:18.

  Average training loss: 0.00
  Accuracy: 1.00
Category: 0
  Precision: 1.00
  Recall: 1.00
  F1: 1.00
Category: 1
  Precision: 0.00
  Recall: 0.00
  F1: 0.00
The av

In [ ]:
all_metrics_one

{0: defaultdict(None,
             {'train_loss_values': [0.02451061978212838,
               0.02451061978212838,
               0.014020035567792874,
               0.014020035567792874,
               0.014816705897016433,
               0.014816705897016433,
               0.012088468786582205,
               0.012088468786582205],
              'train_acc_all': [0.9981796116504854,
               0.9981796116504854,
               0.9981796116504854,
               0.9981796116504854],
              'train_precision_all': [{0: 0.9981796116504854, 1: 0.0},
               {0: 0.9981796116504854, 1: 0.0},
               {0: 0.9981796116504854, 1: 0.0},
               {0: 0.9981796116504854, 1: 0.0}],
              'train_recall_all': [{0: 1.0, 1: 0.0},
               {0: 1.0, 1: 0.0},
               {0: 1.0, 1: 0.0},
               {0: 1.0, 1: 0.0}],
              'train_f1_all': [{0: 0.9990291262135923, 1: 0.0},
               {0: 0.9990291262135923, 1: 0.0},
               {0: 0.99

#### Incremental recitfied training

In [ ]:
import json
batch_size = 32
lr = 3e-5
test_batch = [test_inputs,test_labels,test_masks]
Bert = Bertnn_rectified_info(train_inputs,train_labels,train_masks, validation_inputs,validation_labels,validation_masks,test_batch,batch_size=batch_size,lr=lr)
all_metrics_one, all_tesst_metrics = Bert.searchUpsample(100)
all_metrics_one['test'] = all_tesst_metrics
all_metrics_all_category[section_category] = all_metrics_one
if not os.path.exists('BertSearchResult'):
  os.makedirs('BertSearchResult')

filename = './BertSearchResult/'+'rectified'+'_'+str(section_category)+'_'+str(batch_size) + '_' + str(lr) + '_'  + str(section_category)
with open(filename,'w') as file_obj:
  json.dump(all_metrics_one,file_obj)
  print('Successfully save file %s'%(filename))

del Bert
torch.cuda.empty_cache() 

In [ ]:
all_metrics_one['test']

{0: [[{0: 0.9980769230769231, 1: 0.0}],
  [{0: 1.0, 1: 0.0}],
  [{0: 0.998974358974359, 1: 0.0}],
  [0.9980769230769231]]}

### For 'NewSubEvent'

#### Split

In [ ]:
# Use train_test_split to split our data into train and validation and test sets for
# training
section_category = 0
labels = labels_single[section_category]
from sklearn.model_selection import train_test_split

# Use 80% for training and 20% for validation.
temp_inputs, test_inputs,  temp_labels, test_labels = train_test_split(input_ids, labels, random_state=1999, test_size=0.2)
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(temp_inputs, temp_labels,random_state=1999, test_size=0.2)

# Do the same for the masks.
temp_masks, test_masks, temp_labels, _ = train_test_split(attention_masks, labels,random_state=1999, test_size=0.2)

train_masks, validation_masks, _, _ = train_test_split(temp_masks, temp_labels,random_state=1999, test_size=0.2)

#### Normal training

In [ ]:
import json
batch_size = 16
lr = 2e-5
test_batch = [test_inputs,test_labels,test_masks]
Bert = Bertnn_info(train_inputs,train_labels,train_masks, validation_inputs,validation_labels,validation_masks,test_batch,batch_size=batch_size,lr=lr)
all_metrics_one, all_tesst_metrics = Bert.searchUpsample(100)
all_metrics_one['test'] = all_tesst_metrics
all_metrics_all_category[section_category] = all_metrics_one
if not os.path.exists('BertSearchResult'):
  os.makedirs('BertSearchResult')

filename = './BertSearchResult/'+str(section_category)+'_'+str(batch_size) + '_' + str(lr) + '_'  + str(section_category)
with open(filename,'w') as file_obj:
  json.dump(all_metrics_one,file_obj)
  print('Successfully save file %s'%(filename))

del Bert
torch.cuda.empty_cache() 

There are 1 GPU(s) available.
We will use the GPU: TITAN RTX


I0903 08:16:10.895144 140575546472256 configuration_utils.py:264] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0903 08:16:10.896262 140575546472256 configuration_utils.py:300] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

I0903 08:16:11.016486 140575546472256 modeling_utils.py:667] loading weights file https://

Current increase 0 times
Current is processing category 1
Before up-sample, there are 1648 samples
There are 103 sampled in this category.
After up-sample, there are 1648 samples

Currrent is processing on categorie GoodsServices

Before up-sample, the ratio of current category and all samples
0
0.0

After up-sample, the ratio of current category and all samples
1545
0.9375

Currrent is processing on categorie InformationWanted

Before up-sample, the ratio of current category and all samples
0
0.0

After up-sample, the ratio of current category and all samples
103
0.0625

(1648, 144)


I0903 08:16:14.120489 140575546472256 configuration_utils.py:264] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0903 08:16:14.121343 140575546472256 configuration_utils.py:300] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

I0903 08:16:14.585554 140575546472256 modeling_utils.py:667] loading weights file https://


======== Epoch 1 / 4 ========
Training...
  Batch    40  of    103.    Elapsed: 0:00:08.
  Batch    80  of    103.    Elapsed: 0:00:15.

  Average training loss: 0.26
  Accuracy: 0.93
Category: 0
  Precision: 0.93
  Recall: 0.98
  F1: 0.95
Category: 1
  Precision: 0.00
  Recall: 0.02
  F1: 0.01
The average precision is: 0.4670
The average recall is: 0.5002
The average f1 is: 0.4802
  Training epcoh took: 0:00:19

Running Validation...
  Accuracy: 0.94
Category: 0
  Precision: 0.9412
  Recall: 1.0000
  F1: 0.9689
Category: 1
  Precision: 0.0000
  Recall: 0.0000
  F1: 0.0000
The average precision is: 0.4706
The average recall is: 0.5000
The average f1 is: 0.4845
  Validation took: 0:00:01

======== Epoch 2 / 4 ========
Training...
  Batch    40  of    103.    Elapsed: 0:00:08.
  Batch    80  of    103.    Elapsed: 0:00:15.

  Average training loss: 0.23
  Accuracy: 0.94
Category: 0
  Precision: 0.94
  Recall: 1.00
  F1: 0.97
Category: 1
  Precision: 0.00
  Recall: 0.00
  F1: 0.00
The av

...


======== Epoch 1 / 4 ========
Training...
  Batch    40  of    554.    Elapsed: 0:00:09.
  Batch    80  of    554.    Elapsed: 0:00:18.
  Batch   120  of    554.    Elapsed: 0:00:27.
  Batch   160  of    554.    Elapsed: 0:00:36.
  Batch   200  of    554.    Elapsed: 0:00:46.
  Batch   240  of    554.    Elapsed: 0:00:55.
  Batch   280  of    554.    Elapsed: 0:01:04.
  Batch   320  of    554.    Elapsed: 0:01:14.
  Batch   360  of    554.    Elapsed: 0:01:23.
  Batch   400  of    554.    Elapsed: 0:01:33.
  Batch   440  of    554.    Elapsed: 0:01:42.
  Batch   480  of    554.    Elapsed: 0:01:52.
  Batch   520  of    554.    Elapsed: 0:02:01.

  Average training loss: 0.12
  Accuracy: 0.96
Category: 0
  Precision: 0.82
  Recall: 0.78
  F1: 0.79
Category: 1
  Precision: 0.96
  Recall: 0.99
  F1: 0.98
The average precision is: 0.8922
The average recall is: 0.8851
The average f1 is: 0.8827
  Training epcoh took: 0:02:09

Running Validation...
  Accuracy: 0.94
Category: 0
  Precision: 0

I0903 08:58:21.431536 140575546472256 configuration_utils.py:264] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0903 08:58:21.432586 140575546472256 configuration_utils.py:300] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

I0903 08:58:21.548952 140575546472256 modeling_utils.py:667] loading weights file https://


======== Epoch 1 / 4 ========
Training...
  Batch    40  of    618.    Elapsed: 0:00:08.
  Batch    80  of    618.    Elapsed: 0:00:17.
  Batch   120  of    618.    Elapsed: 0:00:31.
  Batch   160  of    618.    Elapsed: 0:00:41.
  Batch   200  of    618.    Elapsed: 0:00:51.
  Batch   240  of    618.    Elapsed: 0:01:01.
  Batch   280  of    618.    Elapsed: 0:01:10.
  Batch   320  of    618.    Elapsed: 0:01:20.
  Batch   360  of    618.    Elapsed: 0:01:30.
  Batch   400  of    618.    Elapsed: 0:01:39.
  Batch   440  of    618.    Elapsed: 0:01:49.
  Batch   480  of    618.    Elapsed: 0:01:59.
  Batch   520  of    618.    Elapsed: 0:02:09.
  Batch   560  of    618.    Elapsed: 0:02:19.
  Batch   600  of    618.    Elapsed: 0:02:28.

  Average training loss: 0.10
  Accuracy: 0.97
Category: 0
  Precision: 0.87
  Recall: 0.81
  F1: 0.83
Category: 1
  Precision: 0.97
  Recall: 0.99
  F1: 0.98
The average precision is: 0.9211
The average recall is: 0.9044
The average f1 is: 0.9052
  T

I0903 09:08:06.620725 140575546472256 configuration_utils.py:264] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0903 09:08:06.621844 140575546472256 configuration_utils.py:300] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

I0903 09:08:06.700091 140575546472256 modeling_utils.py:667] loading weights file https://


======== Epoch 1 / 4 ========
Training...
  Batch    40  of    683.    Elapsed: 0:00:08.
  Batch    80  of    683.    Elapsed: 0:00:16.
  Batch   120  of    683.    Elapsed: 0:00:25.
  Batch   160  of    683.    Elapsed: 0:00:33.
  Batch   200  of    683.    Elapsed: 0:00:41.
  Batch   240  of    683.    Elapsed: 0:00:49.
  Batch   280  of    683.    Elapsed: 0:00:57.
  Batch   320  of    683.    Elapsed: 0:01:06.
  Batch   360  of    683.    Elapsed: 0:01:14.
  Batch   400  of    683.    Elapsed: 0:01:22.
  Batch   440  of    683.    Elapsed: 0:01:31.
  Batch   480  of    683.    Elapsed: 0:01:39.
  Batch   520  of    683.    Elapsed: 0:01:47.
  Batch   560  of    683.    Elapsed: 0:01:55.
  Batch   600  of    683.    Elapsed: 0:02:04.
  Batch   640  of    683.    Elapsed: 0:02:12.
  Batch   680  of    683.    Elapsed: 0:02:20.

  Average training loss: 0.09
  Accuracy: 0.97
Category: 0
  Precision: 0.79
  Recall: 0.75
  F1: 0.76
Category: 1
  Precision: 0.97
  Recall: 1.00
  F1: 0.9

In [ ]:
all_metrics_one

{0: defaultdict(None,
             {'train_loss_values': [0.02451061978212838,
               0.02451061978212838,
               0.014020035567792874,
               0.014020035567792874,
               0.014816705897016433,
               0.014816705897016433,
               0.012088468786582205,
               0.012088468786582205],
              'train_acc_all': [0.9981796116504854,
               0.9981796116504854,
               0.9981796116504854,
               0.9981796116504854],
              'train_precision_all': [{0: 0.9981796116504854, 1: 0.0},
               {0: 0.9981796116504854, 1: 0.0},
               {0: 0.9981796116504854, 1: 0.0},
               {0: 0.9981796116504854, 1: 0.0}],
              'train_recall_all': [{0: 1.0, 1: 0.0},
               {0: 1.0, 1: 0.0},
               {0: 1.0, 1: 0.0},
               {0: 1.0, 1: 0.0}],
              'train_f1_all': [{0: 0.9990291262135923, 1: 0.0},
               {0: 0.9990291262135923, 1: 0.0},
               {0: 0.99

#### Incremental recitfied training

In [ ]:
import json
batch_size = 32
lr = 3e-5
test_batch = [test_inputs,test_labels,test_masks]
Bert = Bertnn_rectified_info(train_inputs,train_labels,train_masks, validation_inputs,validation_labels,validation_masks,test_batch,batch_size=batch_size,lr=lr)
all_metrics_one, all_tesst_metrics = Bert.searchUpsample(100)
all_metrics_one['test'] = all_tesst_metrics
all_metrics_all_category[section_category] = all_metrics_one
if not os.path.exists('BertSearchResult'):
  os.makedirs('BertSearchResult')

filename = './BertSearchResult/'+'rectified'+'_'+str(section_category)+'_'+str(batch_size) + '_' + str(lr) + '_'  + str(section_category)
with open(filename,'w') as file_obj:
  json.dump(all_metrics_one,file_obj)
  print('Successfully save file %s'%(filename))

del Bert
torch.cuda.empty_cache() 

In [ ]:
all_metrics_one['test']

{0: [[{0: 0.9980769230769231, 1: 0.0}],
  [{0: 1.0, 1: 0.0}],
  [{0: 0.998974358974359, 1: 0.0}],
  [0.9980769230769231]]}

### For 'ServiceAvailable'

#### Split

In [ ]:
# Use train_test_split to split our data into train and validation and test sets for
# training
section_category = 0
labels = labels_single[section_category]
from sklearn.model_selection import train_test_split

# Use 80% for training and 20% for validation.
temp_inputs, test_inputs,  temp_labels, test_labels = train_test_split(input_ids, labels, random_state=1999, test_size=0.2)
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(temp_inputs, temp_labels,random_state=1999, test_size=0.2)

# Do the same for the masks.
temp_masks, test_masks, temp_labels, _ = train_test_split(attention_masks, labels,random_state=1999, test_size=0.2)

train_masks, validation_masks, _, _ = train_test_split(temp_masks, temp_labels,random_state=1999, test_size=0.2)

#### Normal training

In [ ]:
import json
batch_size = 16
lr = 2e-5
test_batch = [test_inputs,test_labels,test_masks]
Bert = Bertnn_info(train_inputs,train_labels,train_masks, validation_inputs,validation_labels,validation_masks,test_batch,batch_size=batch_size,lr=lr)
all_metrics_one, all_tesst_metrics = Bert.searchUpsample(100)
all_metrics_one['test'] = all_tesst_metrics
all_metrics_all_category[section_category] = all_metrics_one
if not os.path.exists('BertSearchResult'):
  os.makedirs('BertSearchResult')

filename = './BertSearchResult/'+str(section_category)+'_'+str(batch_size) + '_' + str(lr) + '_'  + str(section_category)
with open(filename,'w') as file_obj:
  json.dump(all_metrics_one,file_obj)
  print('Successfully save file %s'%(filename))

del Bert
torch.cuda.empty_cache() 

There are 1 GPU(s) available.
We will use the GPU: TITAN RTX


I0903 09:17:37.160173 140575546472256 configuration_utils.py:264] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0903 09:17:37.161261 140575546472256 configuration_utils.py:300] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

I0903 09:17:38.442567 140575546472256 modeling_utils.py:667] loading weights file https://

Current increase 0 times
Current is processing category 1
Before up-sample, there are 1648 samples
There are 87 sampled in this category.
After up-sample, there are 1648 samples

Currrent is processing on categorie GoodsServices

Before up-sample, the ratio of current category and all samples
0
0.0

After up-sample, the ratio of current category and all samples
1561
0.9472087378640777

Currrent is processing on categorie InformationWanted

Before up-sample, the ratio of current category and all samples
0
0.0

After up-sample, the ratio of current category and all samples
87
0.05279126213592233

(1648, 144)


I0903 09:17:41.651647 140575546472256 configuration_utils.py:264] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0903 09:17:41.652540 140575546472256 configuration_utils.py:300] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

I0903 09:17:41.685500 140575546472256 modeling_utils.py:667] loading weights file https://


======== Epoch 1 / 4 ========
Training...
  Batch    40  of    103.    Elapsed: 0:00:08.
  Batch    80  of    103.    Elapsed: 0:00:16.

  Average training loss: 0.24
  Accuracy: 0.94
Category: 0
  Precision: 0.95
  Recall: 0.99
  F1: 0.96
Category: 1
  Precision: 0.00
  Recall: 0.00
  F1: 0.00
The average precision is: 0.4735
The average recall is: 0.4945
The average f1 is: 0.4811
  Training epcoh took: 0:00:21

Running Validation...
  Accuracy: 0.95
Category: 0
  Precision: 0.9471
  Recall: 1.0000
  F1: 0.9720
Category: 1
  Precision: 0.0000
  Recall: 0.0000
  F1: 0.0000
The average precision is: 0.4736
The average recall is: 0.5000
The average f1 is: 0.4860
  Validation took: 0:00:01

======== Epoch 2 / 4 ========
Training...
  Batch    40  of    103.    Elapsed: 0:00:08.
  Batch    80  of    103.    Elapsed: 0:00:16.

  Average training loss: 0.14
  Accuracy: 0.96
Category: 0
  Precision: 0.96
  Recall: 1.00
  F1: 0.98
Category: 1
  Precision: 0.14
  Recall: 0.09
  F1: 0.11
The av

...


======== Epoch 1 / 4 ========
Training...
  Batch    40  of    484.    Elapsed: 0:00:09.
  Batch    80  of    484.    Elapsed: 0:00:18.
  Batch   120  of    484.    Elapsed: 0:00:27.
  Batch   160  of    484.    Elapsed: 0:00:37.
  Batch   200  of    484.    Elapsed: 0:00:46.
  Batch   240  of    484.    Elapsed: 0:00:55.
  Batch   280  of    484.    Elapsed: 0:01:05.
  Batch   320  of    484.    Elapsed: 0:01:14.
  Batch   360  of    484.    Elapsed: 0:01:23.
  Batch   400  of    484.    Elapsed: 0:01:32.
  Batch   440  of    484.    Elapsed: 0:01:42.
  Batch   480  of    484.    Elapsed: 0:01:51.

  Average training loss: 0.10
  Accuracy: 0.97
Category: 0
  Precision: 0.86
  Recall: 0.81
  F1: 0.82
Category: 1
  Precision: 0.97
  Recall: 0.99
  F1: 0.98
The average precision is: 0.9134
The average recall is: 0.9023
The average f1 is: 0.9017
  Training epcoh took: 0:01:51

Running Validation...
  Accuracy: 0.96
Category: 0
  Precision: 0.9636
  Recall: 0.9926
  F1: 0.9772
Category: 1

I0903 09:52:52.720164 140575546472256 configuration_utils.py:264] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0903 09:52:52.721528 140575546472256 configuration_utils.py:300] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

I0903 09:52:53.611905 140575546472256 modeling_utils.py:667] loading weights file https://


======== Epoch 1 / 4 ========
Training...
  Batch    40  of    538.    Elapsed: 0:00:08.
  Batch    80  of    538.    Elapsed: 0:00:16.
  Batch   120  of    538.    Elapsed: 0:00:25.
  Batch   160  of    538.    Elapsed: 0:00:33.
  Batch   200  of    538.    Elapsed: 0:00:41.
  Batch   240  of    538.    Elapsed: 0:00:49.
  Batch   280  of    538.    Elapsed: 0:00:57.
  Batch   320  of    538.    Elapsed: 0:01:06.
  Batch   360  of    538.    Elapsed: 0:01:14.
  Batch   400  of    538.    Elapsed: 0:01:22.
  Batch   440  of    538.    Elapsed: 0:01:30.
  Batch   480  of    538.    Elapsed: 0:01:38.
  Batch   520  of    538.    Elapsed: 0:01:47.

  Average training loss: 0.09
  Accuracy: 0.97
Category: 0
  Precision: 0.82
  Recall: 0.78
  F1: 0.79
Category: 1
  Precision: 0.97
  Recall: 1.00
  F1: 0.98
The average precision is: 0.8920
The average recall is: 0.8899
The average f1 is: 0.8865
  Training epcoh took: 0:01:50

Running Validation...
  Accuracy: 0.94
Category: 0
  Precision: 0

I0903 10:00:27.964368 140575546472256 configuration_utils.py:264] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0903 10:00:27.965423 140575546472256 configuration_utils.py:300] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

I0903 10:00:28.048061 140575546472256 modeling_utils.py:667] loading weights file https://


======== Epoch 1 / 4 ========
Training...
  Batch    40  of    593.    Elapsed: 0:00:08.
  Batch    80  of    593.    Elapsed: 0:00:17.
  Batch   120  of    593.    Elapsed: 0:00:26.
  Batch   160  of    593.    Elapsed: 0:00:34.
  Batch   200  of    593.    Elapsed: 0:00:43.
  Batch   240  of    593.    Elapsed: 0:00:52.
  Batch   280  of    593.    Elapsed: 0:01:01.
  Batch   320  of    593.    Elapsed: 0:01:09.
  Batch   360  of    593.    Elapsed: 0:01:18.
  Batch   400  of    593.    Elapsed: 0:01:27.
  Batch   440  of    593.    Elapsed: 0:01:36.
  Batch   480  of    593.    Elapsed: 0:01:45.
  Batch   520  of    593.    Elapsed: 0:01:54.
  Batch   560  of    593.    Elapsed: 0:02:02.

  Average training loss: 0.07
  Accuracy: 0.98
Category: 0
  Precision: 0.84
  Recall: 0.82
  F1: 0.83
Category: 1
  Precision: 0.98
  Recall: 1.00
  F1: 0.99
The average precision is: 0.9102
The average recall is: 0.9081
The average f1 is: 0.9055
  Training epcoh took: 0:02:09

Running Validation

In [ ]:
all_metrics_one

{0: defaultdict(None,
             {'train_loss_values': [0.02451061978212838,
               0.02451061978212838,
               0.014020035567792874,
               0.014020035567792874,
               0.014816705897016433,
               0.014816705897016433,
               0.012088468786582205,
               0.012088468786582205],
              'train_acc_all': [0.9981796116504854,
               0.9981796116504854,
               0.9981796116504854,
               0.9981796116504854],
              'train_precision_all': [{0: 0.9981796116504854, 1: 0.0},
               {0: 0.9981796116504854, 1: 0.0},
               {0: 0.9981796116504854, 1: 0.0},
               {0: 0.9981796116504854, 1: 0.0}],
              'train_recall_all': [{0: 1.0, 1: 0.0},
               {0: 1.0, 1: 0.0},
               {0: 1.0, 1: 0.0},
               {0: 1.0, 1: 0.0}],
              'train_f1_all': [{0: 0.9990291262135923, 1: 0.0},
               {0: 0.9990291262135923, 1: 0.0},
               {0: 0.99

#### Incremental recitfied training

In [ ]:
import json
batch_size = 32
lr = 3e-5
test_batch = [test_inputs,test_labels,test_masks]
Bert = Bertnn_rectified_info(train_inputs,train_labels,train_masks, validation_inputs,validation_labels,validation_masks,test_batch,batch_size=batch_size,lr=lr)
all_metrics_one, all_tesst_metrics = Bert.searchUpsample(100)
all_metrics_one['test'] = all_tesst_metrics
all_metrics_all_category[section_category] = all_metrics_one
if not os.path.exists('BertSearchResult'):
  os.makedirs('BertSearchResult')

filename = './BertSearchResult/'+'rectified'+'_'+str(section_category)+'_'+str(batch_size) + '_' + str(lr) + '_'  + str(section_category)
with open(filename,'w') as file_obj:
  json.dump(all_metrics_one,file_obj)
  print('Successfully save file %s'%(filename))

del Bert
torch.cuda.empty_cache() 

In [ ]:
all_metrics_one

{0: defaultdict(None,
             {'train_loss_values': [0.02451061978212838,
               0.02451061978212838,
               0.014020035567792874,
               0.014020035567792874,
               0.014816705897016433,
               0.014816705897016433,
               0.012088468786582205,
               0.012088468786582205],
              'train_acc_all': [0.9981796116504854,
               0.9981796116504854,
               0.9981796116504854,
               0.9981796116504854],
              'train_precision_all': [{0: 0.9981796116504854, 1: 0.0},
               {0: 0.9981796116504854, 1: 0.0},
               {0: 0.9981796116504854, 1: 0.0},
               {0: 0.9981796116504854, 1: 0.0}],
              'train_recall_all': [{0: 1.0, 1: 0.0},
               {0: 1.0, 1: 0.0},
               {0: 1.0, 1: 0.0},
               {0: 1.0, 1: 0.0}],
              'train_f1_all': [{0: 0.9990291262135923, 1: 0.0},
               {0: 0.9990291262135923, 1: 0.0},
               {0: 0.99

### For 'Advice'

#### Split

In [ ]:
# Use train_test_split to split our data into train and validation and test sets for
# training
section_category = 0
labels = labels_single[section_category]
from sklearn.model_selection import train_test_split

# Use 80% for training and 20% for validation.
temp_inputs, test_inputs,  temp_labels, test_labels = train_test_split(input_ids, labels, random_state=1999, test_size=0.2)
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(temp_inputs, temp_labels,random_state=1999, test_size=0.2)

# Do the same for the masks.
temp_masks, test_masks, temp_labels, _ = train_test_split(attention_masks, labels,random_state=1999, test_size=0.2)

train_masks, validation_masks, _, _ = train_test_split(temp_masks, temp_labels,random_state=1999, test_size=0.2)

#### Normal training

In [ ]:
import json
batch_size = 16
lr = 2e-5
test_batch = [test_inputs,test_labels,test_masks]
Bert = Bertnn_info(train_inputs,train_labels,train_masks, validation_inputs,validation_labels,validation_masks,test_batch,batch_size=batch_size,lr=lr)
all_metrics_one, all_tesst_metrics = Bert.searchUpsample(100)
all_metrics_one['test'] = all_tesst_metrics
all_metrics_all_category[section_category] = all_metrics_one
if not os.path.exists('BertSearchResult'):
  os.makedirs('BertSearchResult')

filename = './BertSearchResult/'+str(section_category)+'_'+str(batch_size) + '_' + str(lr) + '_'  + str(section_category)
with open(filename,'w') as file_obj:
  json.dump(all_metrics_one,file_obj)
  print('Successfully save file %s'%(filename))

del Bert
torch.cuda.empty_cache() 

  Batch    80  of    228.    Elapsed: 0:00:16.
  Batch   120  of    228.    Elapsed: 0:00:24.
  Batch   160  of    228.    Elapsed: 0:00:32.
  Batch   200  of    228.    Elapsed: 0:00:40.

  Average training loss: 0.44
  Accuracy: 0.79
Category: 0
  Precision: 0.75
  Recall: 0.67
  F1: 0.67
Category: 1
  Precision: 0.81
  Recall: 0.88
  F1: 0.83
The average precision is: 0.7788
The average recall is: 0.7743
The average f1 is: 0.7464
  Training epcoh took: 0:00:45

Running Validation...
  Accuracy: 0.75
Category: 0
  Precision: 0.9277
  Recall: 0.7688
  F1: 0.8353
Category: 1
  Precision: 0.3406
  Recall: 0.6135
  F1: 0.4162
The average precision is: 0.6342
The average recall is: 0.6911
The average f1 is: 0.6258
  Validation took: 0:00:01

======== Epoch 2 / 4 ========
Training...
  Batch    40  of    228.    Elapsed: 0:00:08.
  Batch    80  of    228.    Elapsed: 0:00:16.
  Batch   120  of    228.    Elapsed: 0:00:25.
  Batch   160  of    228.    Elapsed: 0:00:33.
  Batch   200  of    

I0903 13:33:32.426645 140498471425856 configuration_utils.py:264] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0903 13:33:32.428748 140498471425856 configuration_utils.py:300] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

I0903 13:33:32.472086 140498471425856 modeling_utils.py:667] loading weights file https://


======== Epoch 1 / 4 ========
Training...
  Batch    40  of    352.    Elapsed: 0:00:09.
  Batch    80  of    352.    Elapsed: 0:00:18.
  Batch   120  of    352.    Elapsed: 0:00:27.
  Batch   160  of    352.    Elapsed: 0:00:36.
  Batch   200  of    352.    Elapsed: 0:00:45.
  Batch   240  of    352.    Elapsed: 0:00:54.
  Batch   280  of    352.    Elapsed: 0:01:03.
  Batch   320  of    352.    Elapsed: 0:01:14.

  Average training loss: 0.34
  Accuracy: 0.87
Category: 0
  Precision: 0.77
  Recall: 0.56
  F1: 0.61
Category: 1
  Precision: 0.87
  Recall: 0.97
  F1: 0.91
The average precision is: 0.8206
The average recall is: 0.7649
The average f1 is: 0.7604
  Training epcoh took: 0:01:21

Running Validation...
  Accuracy: 0.84
Category: 0
  Precision: 0.9109
  Recall: 0.8985
  F1: 0.9012
Category: 1
  Precision: 0.4487
  Recall: 0.4904
  F1: 0.4382
The average precision is: 0.6798
The average recall is: 0.6944
The average f1 is: 0.6697
  Validation took: 0:00:01

======== Epoch 2 / 4

I0903 13:38:55.621675 140498471425856 configuration_utils.py:264] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0903 13:38:55.623096 140498471425856 configuration_utils.py:300] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

I0903 13:38:56.117106 140498471425856 modeling_utils.py:667] loading weights file https://


======== Epoch 1 / 4 ========
Training...
  Batch    40  of    477.    Elapsed: 0:00:08.
  Batch    80  of    477.    Elapsed: 0:00:17.
  Batch   120  of    477.    Elapsed: 0:00:25.
  Batch   160  of    477.    Elapsed: 0:00:33.
  Batch   200  of    477.    Elapsed: 0:00:41.
  Batch   240  of    477.    Elapsed: 0:00:50.
  Batch   280  of    477.    Elapsed: 0:00:59.
  Batch   320  of    477.    Elapsed: 0:01:07.
  Batch   360  of    477.    Elapsed: 0:01:15.
  Batch   400  of    477.    Elapsed: 0:01:24.
  Batch   440  of    477.    Elapsed: 0:01:32.

  Average training loss: 0.22
  Accuracy: 0.92
Category: 0
  Precision: 0.73
  Recall: 0.64
  F1: 0.66
Category: 1
  Precision: 0.93
  Recall: 0.99
  F1: 0.95
The average precision is: 0.8308
The average recall is: 0.8116
The average f1 is: 0.8048
  Training epcoh took: 0:01:40

Running Validation...
  Accuracy: 0.82
Category: 0
  Precision: 0.8983
  Recall: 0.8946
  F1: 0.8933
Category: 1
  Precision: 0.3872
  Recall: 0.4808
  F1: 0.3

...


======== Epoch 1 / 4 ========
Training...
  Batch    40  of    974.    Elapsed: 0:00:08.
  Batch    80  of    974.    Elapsed: 0:00:16.
  Batch   120  of    974.    Elapsed: 0:00:25.
  Batch   160  of    974.    Elapsed: 0:00:33.
  Batch   200  of    974.    Elapsed: 0:00:42.
  Batch   240  of    974.    Elapsed: 0:00:50.
  Batch   280  of    974.    Elapsed: 0:00:58.
  Batch   320  of    974.    Elapsed: 0:01:07.
  Batch   360  of    974.    Elapsed: 0:01:15.
  Batch   400  of    974.    Elapsed: 0:01:24.
  Batch   440  of    974.    Elapsed: 0:01:32.
  Batch   480  of    974.    Elapsed: 0:01:40.
  Batch   520  of    974.    Elapsed: 0:01:49.
  Batch   560  of    974.    Elapsed: 0:01:57.
  Batch   600  of    974.    Elapsed: 0:02:05.
  Batch   640  of    974.    Elapsed: 0:02:14.
  Batch   680  of    974.    Elapsed: 0:02:22.
  Batch   720  of    974.    Elapsed: 0:02:31.
  Batch   760  of    974.    Elapsed: 0:02:39.
  Batch   800  of    974.    Elapsed: 0:02:47.
  Batch   840  of

I0903 14:30:14.235349 140498471425856 configuration_utils.py:264] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0903 14:30:14.236701 140498471425856 configuration_utils.py:300] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

I0903 14:30:16.434361 140498471425856 modeling_utils.py:667] loading weights file https://


======== Epoch 1 / 4 ========
Training...
  Batch    40  of  1,098.    Elapsed: 0:00:08.
  Batch    80  of  1,098.    Elapsed: 0:00:16.
  Batch   120  of  1,098.    Elapsed: 0:00:25.
  Batch   160  of  1,098.    Elapsed: 0:00:33.
  Batch   200  of  1,098.    Elapsed: 0:00:41.
  Batch   240  of  1,098.    Elapsed: 0:00:50.
  Batch   280  of  1,098.    Elapsed: 0:00:58.
  Batch   320  of  1,098.    Elapsed: 0:01:06.
  Batch   360  of  1,098.    Elapsed: 0:01:14.
  Batch   400  of  1,098.    Elapsed: 0:01:23.
  Batch   440  of  1,098.    Elapsed: 0:01:31.
  Batch   480  of  1,098.    Elapsed: 0:01:40.
  Batch   520  of  1,098.    Elapsed: 0:01:48.
  Batch   560  of  1,098.    Elapsed: 0:01:56.
  Batch   600  of  1,098.    Elapsed: 0:02:05.
  Batch   640  of  1,098.    Elapsed: 0:02:13.
  Batch   680  of  1,098.    Elapsed: 0:02:21.
  Batch   720  of  1,098.    Elapsed: 0:02:30.
  Batch   760  of  1,098.    Elapsed: 0:02:38.
  Batch   800  of  1,098.    Elapsed: 0:02:47.
  Batch   840  of

I0903 14:45:45.599852 140498471425856 configuration_utils.py:264] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0903 14:45:45.601267 140498471425856 configuration_utils.py:300] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

I0903 14:45:46.090362 140498471425856 modeling_utils.py:667] loading weights file https://


======== Epoch 1 / 4 ========
Training...
  Batch    40  of  1,223.    Elapsed: 0:00:08.
  Batch    80  of  1,223.    Elapsed: 0:00:17.
  Batch   120  of  1,223.    Elapsed: 0:00:25.
  Batch   160  of  1,223.    Elapsed: 0:00:33.
  Batch   200  of  1,223.    Elapsed: 0:00:42.
  Batch   240  of  1,223.    Elapsed: 0:00:50.
  Batch   280  of  1,223.    Elapsed: 0:00:59.
  Batch   320  of  1,223.    Elapsed: 0:01:07.
  Batch   360  of  1,223.    Elapsed: 0:01:16.
  Batch   400  of  1,223.    Elapsed: 0:01:24.
  Batch   440  of  1,223.    Elapsed: 0:01:32.
  Batch   480  of  1,223.    Elapsed: 0:01:41.
  Batch   520  of  1,223.    Elapsed: 0:01:49.
  Batch   560  of  1,223.    Elapsed: 0:01:58.
  Batch   600  of  1,223.    Elapsed: 0:02:06.
  Batch   640  of  1,223.    Elapsed: 0:02:14.
  Batch   680  of  1,223.    Elapsed: 0:02:23.
  Batch   720  of  1,223.    Elapsed: 0:02:31.
  Batch   760  of  1,223.    Elapsed: 0:02:39.
  Batch   800  of  1,223.    Elapsed: 0:02:48.
  Batch   840  of

In [ ]:
all_metrics_one

{0: defaultdict(None,
             {'train_loss_values': [0.02451061978212838,
               0.02451061978212838,
               0.014020035567792874,
               0.014020035567792874,
               0.014816705897016433,
               0.014816705897016433,
               0.012088468786582205,
               0.012088468786582205],
              'train_acc_all': [0.9981796116504854,
               0.9981796116504854,
               0.9981796116504854,
               0.9981796116504854],
              'train_precision_all': [{0: 0.9981796116504854, 1: 0.0},
               {0: 0.9981796116504854, 1: 0.0},
               {0: 0.9981796116504854, 1: 0.0},
               {0: 0.9981796116504854, 1: 0.0}],
              'train_recall_all': [{0: 1.0, 1: 0.0},
               {0: 1.0, 1: 0.0},
               {0: 1.0, 1: 0.0},
               {0: 1.0, 1: 0.0}],
              'train_f1_all': [{0: 0.9990291262135923, 1: 0.0},
               {0: 0.9990291262135923, 1: 0.0},
               {0: 0.99

#### Incremental recitfied training

In [ ]:
import json
batch_size = 32
lr = 3e-5
test_batch = [test_inputs,test_labels,test_masks]
Bert = Bertnn_rectified_info(train_inputs,train_labels,train_masks, validation_inputs,validation_labels,validation_masks,test_batch,batch_size=batch_size,lr=lr)
all_metrics_one, all_tesst_metrics = Bert.searchUpsample(100)
all_metrics_one['test'] = all_tesst_metrics
all_metrics_all_category[section_category] = all_metrics_one
if not os.path.exists('BertSearchResult'):
  os.makedirs('BertSearchResult')

filename = './BertSearchResult/'+'rectified'+'_'+str(section_category)+'_'+str(batch_size) + '_' + str(lr) + '_'  + str(section_category)
with open(filename,'w') as file_obj:
  json.dump(all_metrics_one,file_obj)
  print('Successfully save file %s'%(filename))

del Bert
torch.cuda.empty_cache() 

In [ ]:
all_metrics_one

{0: defaultdict(None,
             {'train_loss_values': [0.02451061978212838,
               0.02451061978212838,
               0.014020035567792874,
               0.014020035567792874,
               0.014816705897016433,
               0.014816705897016433,
               0.012088468786582205,
               0.012088468786582205],
              'train_acc_all': [0.9981796116504854,
               0.9981796116504854,
               0.9981796116504854,
               0.9981796116504854],
              'train_precision_all': [{0: 0.9981796116504854, 1: 0.0},
               {0: 0.9981796116504854, 1: 0.0},
               {0: 0.9981796116504854, 1: 0.0},
               {0: 0.9981796116504854, 1: 0.0}],
              'train_recall_all': [{0: 1.0, 1: 0.0},
               {0: 1.0, 1: 0.0},
               {0: 1.0, 1: 0.0},
               {0: 1.0, 1: 0.0}],
              'train_f1_all': [{0: 0.9990291262135923, 1: 0.0},
               {0: 0.9990291262135923, 1: 0.0},
               {0: 0.99